In [1]:
# not use MAPIE
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

iris = pd.read_csv('iris.csv')
iris.sample(3)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
136,6.3,3.4,5.6,2.4,virginica
5,5.4,3.9,1.7,0.4,setosa
130,7.4,2.8,6.1,1.9,virginica


In [2]:
# perform label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# transform the string labels to integer
iris['Species'] = le.fit_transform(iris['Species'].to_numpy())
iris.sample(10)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
111,6.4,2.7,5.3,1.9,2
28,5.2,3.4,1.4,0.2,0
58,6.6,2.9,4.6,1.3,1
94,5.6,2.7,4.2,1.3,1
6,4.6,3.4,1.4,0.3,0
65,6.7,3.1,4.4,1.4,1
82,5.8,2.7,3.9,1.2,1
71,6.1,2.8,4.0,1.3,1
132,6.4,2.8,5.6,2.2,2
97,6.2,2.9,4.3,1.3,1


In [3]:
# input data
X = iris.drop(columns=['Species'])
# output data
Y = iris['Species']

In [4]:
seed = 123
# data split
from sklearn.model_selection import train_test_split
## train - 80%
## test - 20%
# split the data for train, calibration and test
x_train, x_remain, y_train, y_remain = train_test_split(X, Y, test_size=0.25, random_state=seed)
x_cal, x_test, y_cal, y_test = train_test_split(x_remain, y_remain, test_size=0.50, random_state=seed)
# x_train.describe(), x_cal.describe(), x_test.describe()
y_cal

46     0
37     0
20     0
0      0
131    2
31     0
135    2
130    2
90     1
104    2
93     1
72     1
112    2
24     0
45     0
13     0
62     1
117    2
132    2
Name: Species, dtype: int64

In [5]:
# build a prediction model

# knn - k-nearest neighbours
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()

# decision tree
# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier()
model.fit(x_train, y_train)

KNeighborsClassifier()

In [10]:
# predict on calibration data
n = len(x_cal)
preds = model.predict_proba(x_cal)
prob_true = preds[np.arange(n), y_cal]
scores = 1 - prob_true
# n, preds, prob_true, scores
preds, scores

(array([[1. , 0. , 0. ],
        [1. , 0. , 0. ],
        [1. , 0. , 0. ],
        [1. , 0. , 0. ],
        [0. , 0. , 1. ],
        [1. , 0. , 0. ],
        [0. , 0. , 1. ],
        [0. , 0. , 1. ],
        [0. , 1. , 0. ],
        [0. , 0. , 1. ],
        [0. , 1. , 0. ],
        [0. , 0.4, 0.6],
        [0. , 0. , 1. ],
        [1. , 0. , 0. ],
        [1. , 0. , 0. ],
        [1. , 0. , 0. ],
        [0. , 1. , 0. ],
        [0. , 0. , 1. ],
        [0. , 0. , 1. ]]),
 array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. ,
        0. , 0. , 0. , 0. , 0. , 0. ]))

In [96]:
alpha = 0.05
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(scores, q_level, method='higher')
prediction_sets = (1 - model.predict_proba(x_test) <= qhat)

real = le.classes_[iris.Species[y_test.index]]
predset = ['']*len(y_test)
coverage = [0]*len(y_test)
for i in range(len(y_test)):
    predset[i] = ','.join(le.classes_[prediction_sets[i]])
    if real[i] in predset[i]:
        coverage[i] = 1
qhat, 1-model.predict_proba(x_test), prediction_sets

(0.6,
 array([[1. , 0. , 1. ],
        [1. , 1. , 0. ],
        [1. , 0. , 1. ],
        [1. , 1. , 0. ],
        [1. , 0.6, 0.4],
        [0. , 1. , 1. ],
        [1. , 0.2, 0.8],
        [1. , 0.6, 0.4],
        [1. , 1. , 0. ],
        [0. , 1. , 1. ],
        [0. , 1. , 1. ],
        [1. , 0.6, 0.4],
        [1. , 0.6, 0.4],
        [0. , 1. , 1. ],
        [0. , 1. , 1. ],
        [0. , 1. , 1. ],
        [1. , 0. , 1. ],
        [0. , 1. , 1. ],
        [0. , 1. , 1. ]]),
 array([[False,  True, False],
        [False, False,  True],
        [False,  True, False],
        [False, False,  True],
        [False,  True,  True],
        [ True, False, False],
        [False,  True, False],
        [False,  True,  True],
        [False, False,  True],
        [ True, False, False],
        [ True, False, False],
        [False,  True,  True],
        [False,  True,  True],
        [ True, False, False],
        [ True, False, False],
        [ True, False, False],
        [False,  True

In [69]:
# result
res = pd.DataFrame(list(zip(real, predset, coverage)),
                  columns=['actual', 'predicted', 'coverage'])
res

,actual,predicted,coverage
0,versicolor,versicolor,1
1,virginica,virginica,1
2,versicolor,versicolor,1
3,virginica,virginica,1
4,virginica,"versicolor,virginica",1
5,setosa,setosa,1
6,versicolor,versicolor,1
7,virginica,"versicolor,virginica",1
8,virginica,virginica,1
9,setosa,setosa,1


In [70]:
# calculate cover rate
print("Cover rate =", sum(res.coverage)/len(res))

Cover rate = 1.0


In [71]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
# print metric to get performance
print("Accuracy: ", model.score(x_test, y_test))
yhat = model.predict(x_test)

Accuracy:  1.0


In [72]:
conf_matrix = confusion_matrix(y_true=y_cal, y_pred=model.predict(x_cal))
conf_matrix

array([[8, 0, 0],
       [0, 3, 1],
       [0, 0, 7]])